# Online Hardware Validation

Interactive notebook for running **live** hardware validation tests.

**Workflow:**
1. Run the Setup cells once
2. Edit Configuration as needed
3. Run test cells one by one, review outputs, move to next

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# IMPORTS (run once)
# ═══════════════════════════════════════════════════════════════════════════════
import asyncio
import json
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

import nest_asyncio
nest_asyncio.apply()

from acquisition import HardwareClient, capture_live_stream
from analysis.pipeline import run_pipeline
from analysis.decode_bin import parse_framestream_bin
from analysis.plots import plot_eo_ec_publication_complete

print("✓ Imports ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION - EDIT THESE BEFORE RUNNING TESTS
# ═══════════════════════════════════════════════════════════════════════════════

# ─── Connection ────────────────────────────────────────────────────────────────
ESP32_IP = "node.local"           # ESP32 hostname or IP address

# ─── Recording ─────────────────────────────────────────────────────────────────
DURATION_S = 5.0                  # Capture duration in seconds

# ─── Hardware Parameters ───────────────────────────────────────────────────────
SAMPLING_RATE_HZ = 16000          # ADS1299: 250, 500, 1000, 2000, 4000, 8000, 16000
GAIN = 24                         # PGA gain: 1, 2, 4, 6, 8, 12, 24
MAINS_FREQ_HZ = 50.0              # Notch filter: 50 Hz (EU) or 60 Hz (US)

# ─── Metadata ──────────────────────────────────────────────────────────────────
FIRMWARE = "FW2"                  # Firmware version
BOARD = "R2"                      # Board revision

# ─── Output ────────────────────────────────────────────────────────────────────
SAVE_FILES = True                 # Save .bin + sidecar .json
EXPORT_PDF = True                 # Generate PDF reports
OUTPUT_DIR = Path("data/live")    # Output directory

# ─── Hardware Profiles ─────────────────────────────────────────────────────────
PROFILES = {
    "INT": "acquisition/profiles/all_shorted.json",
    "EXT": "acquisition/profiles/all_normal.json",
    "KSI": "acquisition/profiles/test_signal.json",
    "FUN": "acquisition/profiles/eyes_open_closed.json",
}

# ─── Connect to Hardware ───────────────────────────────────────────────────────
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
client = HardwareClient(ESP32_IP)

# Try to detect sample rate from hardware
detected = client.detect_sample_rate()
if detected:
    print(f"✓ Connected to {ESP32_IP}")
    if detected != SAMPLING_RATE_HZ:
        print(f"  ⚠ Detected {detected} Hz (configured: {SAMPLING_RATE_HZ} Hz)")
        SAMPLING_RATE_HZ = detected
    print(f"  Sample rate: {SAMPLING_RATE_HZ} Hz")
else:
    print(f"⚠ Could not detect sample rate, using: {SAMPLING_RATE_HZ} Hz")

print(f"  Duration: {DURATION_S}s | Gain: {GAIN}x | Mains: {MAINS_FREQ_HZ} Hz")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# HELPER FUNCTION (run once)
# ═══════════════════════════════════════════════════════════════════════════════

def capture_and_analyze(test_name, condition_code, profile_path=None):
    """Capture data, save file, run analysis, return results."""
    
    print(f"\n{'='*60}")
    print(f" {test_name} ".center(60, "="))
    print(f"{'='*60}\n")
    
    # Apply profile if provided
    if profile_path:
        print(f"[PROFILE] {Path(profile_path).name}")
        if client.load_and_apply_profile(profile_path):
            client.dump_and_verify_registers()
        else:
            print("⚠ Profile failed, using current settings")
    
    # Capture
    _, raw_frames, _ = asyncio.run(
        capture_live_stream(client, DURATION_S, SAMPLING_RATE_HZ)
    )
    
    if len(raw_frames) == 0:
        print("❌ No frames received")
        return None, None
    
    print(f"✓ Captured {len(raw_frames):,} bytes")
    
    bin_path = None
    if SAVE_FILES:
        n_bytes = len(raw_frames)
        assert n_bytes % 1416 == 0, f"Bad frame alignment: {n_bytes}"
        n_frames = n_bytes // 1416
        
        now = datetime.now()
        filename = f"{now.strftime('%y%m%d_%H%M%S')}_{FIRMWARE}_{BOARD}_{condition_code}_{n_frames}_{SAMPLING_RATE_HZ//1000}k.bin"
        bin_path = OUTPUT_DIR / filename
        bin_path.write_bytes(raw_frames)
        print(f"✓ Saved: {bin_path.name}")
        
        # Sidecar JSON
        sidecar = {
            "timestamp": now.isoformat(),
            "esp32_ip": ESP32_IP,
            "firmware": FIRMWARE,
            "board": BOARD,
            "condition_code": condition_code,
            "profile_path": profile_path,
            "duration_s": DURATION_S,
            "fs_hz": SAMPLING_RATE_HZ,
            "gain": GAIN,
            "mains_freq_hz": MAINS_FREQ_HZ,
            "n_frames": n_frames,
            "n_bytes": n_bytes,
        }
        bin_path.with_suffix('.json').write_text(json.dumps(sidecar, indent=2))
    
    # Analyze
    if SAVE_FILES and bin_path:
        results = run_pipeline(
            bin_path,
            display_plots=True,
            export_pdf_report=EXPORT_PDF,
            fs_hz=SAMPLING_RATE_HZ,
            gain=GAIN,
            mains_freq_hz=MAINS_FREQ_HZ,
        )
    else:
        results = run_pipeline(
            raw_frames,
            source_name=test_name,
            display_plots=True,
            fs_hz=SAMPLING_RATE_HZ,
            gain=GAIN,
            mains_freq_hz=MAINS_FREQ_HZ,
        )
    
    print(f"\n✓ {test_name} complete")
    return results, bin_path

print("✓ Helper function ready")

---
## Internal Noise (Shorted Inputs)

In [ ]:
results_internal, bin_internal = capture_and_analyze("Internal Noise", "INT", PROFILES["INT"])

---
## External Noise (Floating Inputs)

In [ ]:
results_external, bin_external = capture_and_analyze("External Noise", "EXT", PROFILES["EXT"])

---
## Known Signal Injection (KSI) - Channel 2

In [ ]:
results_ksi_ch2, _ = capture_and_analyze("KSI 40Hz CH2", "KSI_CH2_40HZ", PROFILES["KSI"])

## KSI - Channels 3-8

In [ ]:
results_ksi_ch3, _ = capture_and_analyze("KSI 40Hz CH3", "KSI_CH3_40HZ", PROFILES["KSI"])

In [ ]:
results_ksi_ch4, _ = capture_and_analyze("KSI 40Hz CH4", "KSI_CH4_40HZ", PROFILES["KSI"])

In [ ]:
results_ksi_ch5, _ = capture_and_analyze("KSI 40Hz CH5", "KSI_CH5_40HZ", PROFILES["KSI"])

In [ ]:
results_ksi_ch6, _ = capture_and_analyze("KSI 40Hz CH6", "KSI_CH6_40HZ", PROFILES["KSI"])

In [ ]:
results_ksi_ch7, _ = capture_and_analyze("KSI 40Hz CH7", "KSI_CH7_40HZ", PROFILES["KSI"])

In [ ]:
results_ksi_ch8, _ = capture_and_analyze("KSI 40Hz CH8", "KSI_CH8_40HZ", PROFILES["KSI"])

---
## Functional EEG - Eyes Open

In [ ]:
input("Subject: Keep eyes OPEN. Press Enter to start...")
results_eo, bin_eo = capture_and_analyze("Eyes Open", "FUNEO", PROFILES["FUN"])

## Functional EEG - Eyes Closed

In [ ]:
input("Subject: Keep eyes CLOSED. Press Enter to start...")
results_ec, bin_ec = capture_and_analyze("Eyes Closed", "FUNEC", PROFILES["FUN"])

## Functional EEG - Eye Blinks

In [ ]:
input("Subject: Blink every 2-3 seconds. Press Enter to start...")
results_blink, _ = capture_and_analyze("Eye Blinks", "FUNEB", PROFILES["FUN"])

---
## Eyes Open vs Closed Comparison

In [ ]:
if bin_eo and bin_ec and bin_eo.exists() and bin_ec.exists():
    print("Generating EO vs EC comparison...")
    
    counts_eo, meta_eo = parse_framestream_bin(bin_eo.read_bytes())
    counts_ec, meta_ec = parse_framestream_bin(bin_ec.read_bytes())
    
    fig = plot_eo_ec_publication_complete(
        counts_eo, counts_ec,
        fs_hz=SAMPLING_RATE_HZ,
        duration_s=5.0,
        psd_channel=5,
        montage_image_path="analysis/assets/montage_head.png",
    )
    plt.show()
else:
    print("⚠ Run Eyes Open and Eyes Closed tests first")

---
## Custom Test

In [ ]:
# Edit these and run:
TEST_NAME = "Custom Test"
CONDITION = "CUSTOM"
PROFILE = None  # or PROFILES["INT"], PROFILES["KSI"], etc.

results_custom, _ = capture_and_analyze(TEST_NAME, CONDITION, PROFILE)